In [22]:
import pandas as pd

# Read the results file into a DataFrame
def analyze(fname):
    df = pd.read_csv(fname, delimiter='\t')

    # Calculate average metrics for all age groups
    avg_metrics_all_age_groups = df.groupby("Split Name")[["MAE", "RMSE", "MAPE"]].mean()
    
    
    # Print average metrics for all age groups
    print("Average Metrics for All Age Groups:")
    print(avg_metrics_all_age_groups)

    for metric in ["MAE", "RMSE", "MAPE"]:
        # Find the top N approaches based on MAE (you can change N as needed)
        top_n_approaches = df.groupby(["Bandpass", "Detector", "Smoothing"])[metric].mean().sort_values().head(10)
        print(f"\nTop N Approaches ({metric}):")
        print(top_n_approaches)
    
    # Calculate metrics for the top N approaches within each age group
    for split_name in df["Split Name"].unique():
        split_df = df[df["Split Name"] == split_name]
        top_n_approaches_split = split_df.groupby(["Bandpass", "Detector", "Smoothing"])["MAE"].mean().sort_values().head(10)
        
        print(f"\nTop N Approaches for {split_name}:")
        print(top_n_approaches_split)

In [23]:
analyze("age_rr.tsv")

Average Metrics for All Age Groups:
                  MAE      RMSE       MAPE
Split Name                                
Ages 0-50    1.968001  3.092013  12.636279
Ages 50-60   1.438419  2.194996   9.794641
Ages 60-70   2.032513  3.078888  14.959858
Ages 70-80   2.082418  3.300906  12.807915
Ages 80-100  1.977782  2.959292  11.346971

Top N Approaches (MAE):
Bandpass     Detector  Smoothing     
bessel       peak      moving_average    1.391680
                       median            1.418548
elliptic     peak      moving_average    1.442438
butterworth  peak      moving_average    1.450884
chebyshev    peak      moving_average    1.468788
bessel       zcr       moving_average    1.478365
elliptic     peak      median            1.481530
chebyshev    peak      median            1.497445
butterworth  peak      median            1.505781
bessel       mcr       moving_average    1.524770
Name: MAE, dtype: float64

Top N Approaches (RMSE):
Bandpass     Detector  Smoothing     
bessel    

In [7]:
analyze("gender_rr.tsv")

Average Metrics for All Age Groups:
                 MAE      RMSE       MAPE
Split Name                               
Female      2.151676  3.314546  14.234953
Male        1.546609  2.496033   9.209084

Top N Approaches:
Bandpass     Detector  Smoothing     
bessel       peak      moving_average    1.436769
elliptic     peak      moving_average    1.463949
bessel       peak      median            1.468867
butterworth  peak      moving_average    1.477491
bessel       zcr       moving_average    1.477813
chebyshev    peak      moving_average    1.485053
elliptic     peak      median            1.493919
chebyshev    peak      median            1.503430
bessel       mcr       moving_average    1.524966
                       median            1.536465
Name: MAE, dtype: float64

Top N Approaches for Male:
Bandpass     Detector  Smoothing     
bessel       zcr       moving_average    1.247129
             mcr       moving_average    1.301874
                       median            1.3019

In [8]:
analyze("gender_hr.tsv")

Average Metrics for All Age Groups:
                 MAE       RMSE      MAPE
Split Name                               
Female      3.589737   9.551957  3.450920
Male        4.235111  10.951933  3.934309

Top N Approaches:
Bandpass     Detector  Smoothing     
butterworth  zcr       moving_average    1.329518
             mcr       moving_average    1.333208
chebyshev    mcr       moving_average    1.480957
elliptic     zcr       moving_average    1.481204
             mcr       moving_average    1.486596
chebyshev    zcr       moving_average    1.491482
elliptic     spectral  moving_average    1.515970
chebyshev    spectral  moving_average    1.515970
butterworth  mcr       median            1.564927
             spectral  moving_average    1.568033
Name: MAE, dtype: float64

Top N Approaches for Male:
Bandpass     Detector  Smoothing     
bessel       spectral  median            1.515142
butterworth  mcr       moving_average    1.527803
             zcr       moving_average    1.5291

In [29]:
import pandas as pd

# Read the results file into a DataFrame
df = pd.read_csv("gender_rr.tsv", delimiter='\t')

# Mapping for human-readable column names
column_mapping = {
    "Bandpass": "Bandpass Filter",
    "Detector": "Detector",
    "Smoothing": "Smoothing Filter",
    "Split Name": "Age Group",
    "MAE": "Mean Absolute Error",
    "RMSE": "Root Mean Squared Error",
    "MAPE": "Mean Absolute Percentage Error"
}

# Rename columns for human readability
df = df.rename(columns=column_mapping)

# Function to calculate rank for a metric
def calculate_rank(metric_values):
    return metric_values.rank().astype(int)

N = 8

# Calculate ranks for each metric
df["MAE Rank"] = calculate_rank(df["Mean Absolute Error"])
df["RMSE Rank"] = calculate_rank(df["Root Mean Squared Error"])
df["MAPE Rank"] = calculate_rank(df["Mean Absolute Percentage Error"])

# Calculate overall rank based on the sum of ranks
df["Overall Rank"] = df[["MAE Rank", "RMSE Rank", "MAPE Rank"]].sum(axis=1)

# Find the top N approaches based on overall rank (you can change N as needed)
top_n_approaches = df.sort_values("Overall Rank").head(N)

# Print top N approaches
print("Top N Approaches based on Overall Rank:")
print(top_n_approaches[["Bandpass Filter", "Detector", "Smoothing Filter", "Age Group"]])

# Calculate metrics for the top N approaches within each age group
for split_name in df["Age Group"].unique():
    split_df = df[df["Age Group"] == split_name]
    top_n_approaches_split = split_df.sort_values("Overall Rank").head(N)
    
    print(f"\nTop N Approaches for {split_name}:")
    print(top_n_approaches_split[["Bandpass Filter", "Detector", "Smoothing Filter"]])


Top N Approaches based on Overall Rank:
   Bandpass Filter Detector Smoothing Filter Age Group
30          bessel      zcr   moving_average      Male
50          bessel      zcr              NaN      Male
60          bessel      mcr   moving_average      Male
70          bessel      mcr           median      Male
28        elliptic     peak              NaN      Male
80          bessel      mcr              NaN      Male
8         elliptic     peak   moving_average      Male
4        chebyshev     peak   moving_average      Male

Top N Approaches for Male:
   Bandpass Filter Detector Smoothing Filter
30          bessel      zcr   moving_average
50          bessel      zcr              NaN
60          bessel      mcr   moving_average
70          bessel      mcr           median
80          bessel      mcr              NaN
28        elliptic     peak              NaN
8         elliptic     peak   moving_average
4        chebyshev     peak   moving_average

Top N Approaches for Female:
  

In [51]:
import pandas as pd

# Read the results file into a DataFrame
df = pd.read_csv("age_hr.tsv", delimiter='\t')

# Mapping for human-readable column names
column_mapping = {
    "Bandpass": "Bandpass Filter",
    "Detector": "Detector",
    "Smoothing": "Smoothing Filter",
    "Split Name": "Age Group",
    "MAE": "Mean Absolute Error",
    "RMSE": "Root Mean Squared Error",
    "MAPE": "Mean Absolute Percentage Error"
}

# Rename columns for human readability
df = df.rename(columns=column_mapping)

# Function to calculate rank for a metric
def calculate_rank(metric_values):
    return metric_values.rank().astype(int)

N = 8

# Calculate ranks for each metric within each age group
df["MAE Rank"] = df.groupby("Age Group")["Mean Absolute Error"].rank().astype(int)
df["RMSE Rank"] = df.groupby("Age Group")["Root Mean Squared Error"].rank().astype(int)
df["MAPE Rank"] = df.groupby("Age Group")["Mean Absolute Percentage Error"].rank().astype(int)

# Calculate overall rank based on the sum of ranks
df["Overall Rank"] = df[["MAE Rank", "RMSE Rank", "MAPE Rank"]].sum(axis=1)

# Sum the overall ranks for different genders with the same setup
df["Overall Rank Sum"] = df.groupby(["Bandpass Filter", "Detector", "Smoothing Filter"])["Overall Rank"].transform("sum")

# Find the top N approaches based on the sum of overall ranks
top_n_approaches = df.sort_values("Overall Rank Sum").head(N)

# Print top N approaches
print("Top N Approaches based on Overall Rank Sum:")
print(top_n_approaches[["Bandpass Filter", "Detector", "Smoothing Filter", "Overall Rank Sum", "Age Group"]])

# Calculate metrics for the top N approaches within each age group
for split_name in df["Age Group"].unique():
    split_df = df[df["Age Group"] == split_name]
    top_n_approaches_split = split_df.sort_values("Overall Rank").head(N)
    
    print(f"\nTop N Approaches for {split_name}:")
    print(top_n_approaches_split[["Bandpass Filter", "Detector", "Smoothing Filter", "Overall Rank"]])


Top N Approaches based on Overall Rank Sum:
    Bandpass Filter Detector Smoothing Filter  Overall Rank Sum    Age Group
82      butterworth      zcr   moving_average             141.0   Ages 60-70
80      butterworth      zcr   moving_average             141.0    Ages 0-50
84      butterworth      zcr   moving_average             141.0  Ages 80-100
81      butterworth      zcr   moving_average             141.0   Ages 50-60
83      butterworth      zcr   moving_average             141.0   Ages 70-80
158     butterworth      mcr   moving_average             144.0   Ages 70-80
155     butterworth      mcr   moving_average             144.0    Ages 0-50
159     butterworth      mcr   moving_average             144.0  Ages 80-100

Top N Approaches for Ages 0-50:
    Bandpass Filter  Detector Smoothing Filter  Overall Rank
155     butterworth       mcr   moving_average             3
80      butterworth       zcr   moving_average             6
95         elliptic       zcr   moving_average 

In [42]:
len(df.groupby("Age Group")["Mean Absolute Error"])

5